In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
roi = ee.FeatureCollection('users/311605001111/caohu_region')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

# # 青海省部分地区
# roi = ee.Geometry.Rectangle([89.852623, 34.537942,92.3677, 36.217184])
# Map.addLayer(roi, {}, "roi")
# Map.centerObject(roi,7)

In [ ]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(awei)

def area_after_clip(image):
    image_area = image.clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})

In [ ]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(water_index)
#     .map(maskL8sr) \
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')
print(landsat_images.size().getInfo())

In [ ]:
# image = ee.Image(waterpixel_image.toList(waterpixel_image.size()).get(5))
# Map.addLayer(image,{'palette':['white','blue'],'min':0,'max':1},'image5')

image = ee.Image(landsat_images.toList(landsat_images.size()).get(5)).clip(roi)
Map.addLayer(image,visParams,'raw image5')

# 筛选得出采样影像

In [ ]:
image = landsat_images.mosaic().clip(roi)
image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14
})
# print(areas.getInfo())

# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('area',ee.Number(areas.get('B2')).divide(ee.Number(10))))
dates = sample_image.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
print(dates.getInfo())

# 样本采集函数的定义

In [ ]:
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
Map.addLayer(index_waterfrequency.gte(0.75).selfMask(),{'palette':['green']},"index permanentwater")
# JRC
JRC_2018 = ee.Image("JRC/GSW1_3/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# # Maryland
# Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(roi).select('b1').rename('waterclass')
# Maryland_permanent = Maryland_2018.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
# Maryland_season= ee.ImageCollection([Maryland_2018.gte(25),Maryland_2018.lt(75)]).sum().eq(2)
# Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# # 三者的交集
# permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
# landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
# seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
# 两者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

Map.addLayer(permanentWaterExtent.selfMask(),{'palette':['blue']},"permanent_water")
Map.addLayer(seasonWaterExtent.selfMask(),{'palette':['green']},"season water")

In [ ]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi','AWEI']
# 分类标签
label = 'waterclass'
def imageSample(image):
    ROI = image.clip(roi).geometry()
    permanent = permanentWaterExtent.updateMask(image.select('B3').gt(0)).remap([0,1],[0,2]).rename('waterclass')
    land = landExtent.updateMask(image.select('B3').gt(0))
    awei = image.clip(roi).updateMask(seasonWaterExtent).select('AWEI')
    intervalMean = awei.reduceRegion(**{
        'reducer': ee.Reducer.intervalMean(50,100), 
        'geometry': roi, 
        'scale': 30, 
        'bestEffort': True
    })
    season_class = awei.gt(ee.Image.constant(intervalMean.get('AWEI'))).rename('waterclass')
    seasonalwater_Extent = ee.ImageCollection([season_class,basemap]).sum().remap([0,1],[0,2]).rename('waterclass').clip(image.geometry())
    percentile_waterclass = ee.ImageCollection([permanent,seasonalwater_Extent,land]).sum().remap([0,1,2],[0,0,1]).rename('waterclass')
    water = percentile_waterclass.selfMask().clip(image.geometry())
    nowater = percentile_waterclass.eq(0).selfMask().remap([1],[0]).rename('waterclass').clip(image.geometry())
    water_points = water.sample(**{
        'region': ROI,
        'scale': 30,
        'numPixels': 200,
        'seed': 1,
        'geometries': True,
    })
    nowater_points = nowater.sample(**{
        'region': ROI,
        'scale': 30,
        'numPixels': 100,
        'seed': 1,
        'geometries': True,
    })
    all_points = water_points.merge(nowater_points)
    #.merge(nowater_points)
    sample_points = image.select(bands).sampleRegions(**{
        'collection': all_points,
        'properties': [label],
        'scale': 30,
        'geometries': True,
    })
    return sample_points

In [ ]:
# 总的样本点数
points_collection = sample_image.map(imageSample).flatten()
print(points_collection.first().getInfo())
print('total sample number:{}'.format(points_collection.size().getInfo()))

# # 水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

# # 非水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',0))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

In [ ]:
def AutomaticCorrection(image):
    basemap = ee.Image.constant(0).toFloat().clip(image.geometry()).clip(roi).rename('waterclass')
    class_image = ee.ImageCollection([basemap,image.remap([0,1],[1,2]).rename('waterclass')]).sum()
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence').clip(image.geometry()).clip(roi)
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': image.clip(roi).geometry(),
        'scale': 90,
    })
    HistogramCount_sum = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram')).reduce(ee.Reducer.sum())
    count_threshold = ee.Number(HistogramCount_sum).multiply(0.0017)
    occurrence_histogram = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.autoHistogram(100,1),
        'geometry': image.clip(roi).geometry(),
        'scale': 90,
    })
    occurrence_list = ee.List(occurrence_histogram.get('occurrence').getInfo())
    occurrence_threshold = occurrence_list.map(lambda array : ee.Algorithms.If(ee.Number(ee.List(array).get(1)).gte(ee.Number(count_threshold)),ee.List(array).get(0))).removeAll([None]).get(0)
    print(occurrence_threshold.getInfo())
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water

In [ ]:
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

#对影像进行分类
def training(image):
    return image.clip(roi).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')

In [ ]:
waterpixel_image = sample_image.map(training)

# print(waterpixel_image.size().getInfo())

In [ ]:
# basemap = ee.Image.constant(0).toFloat().clip(roi).rename('waterclass')
imageCol = ee.ImageCollection([])
for i in range(sample_image.size().getInfo()):
    image = ee.Image(waterpixel_image.toList(waterpixel_image.size()).get(i))
    correction_image = AutomaticCorrection(image)
    imageCol = imageCol.merge(ee.ImageCollection([correction_image]))
# dates = imageCol.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
# print(imageCol.size().getInfo())

In [ ]:
dates = imageCol.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(dates.getInfo())

In [ ]:
image = classify_image.remap([0,1],[1,2]).rename('waterclass')
class_image = ee.ImageCollection([basemap,image]).sum()
# Map.addLayer(class_image,{'palette':['white','red','blue'],'min':0,'max':2},'class image')
water = class_image.eq(2).selfMask()
no_data = class_image.eq(0).selfMask()

# 加载water occurrence 制图产品
visualization_occurrence = {
  'bands': ['occurrence'],
  'min': 0.0,
  'max': 100.0,
  'palette': ['green', 'yellow', 'red']
}
occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence').clip(image9.geometry()).clip(roi)
occurrence_water = occurrence.updateMask(water)
occurrence_no_data = occurrence.updateMask(no_data)
# Map.addLayer(occurrence_water, visualization_occurrence, 'water in occurrence')
# Map.addLayer(occurrence_no_data, visualization_occurrence, 'no data in occurrence')

In [ ]:
# 计算 count threshold ；count_threshold = "The average value of pixel counts" *0.17
occurrence_HistogramCount = occurrence_water.reduceRegion(**{
    'reducer': ee.Reducer.histogram(100,1),
    'geometry': image9.clip(roi).geometry(),
    'scale': 60,
})
HistogramCount_sum = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram')).reduce(ee.Reducer.sum())
count_threshold = ee.Number(HistogramCount_sum).multiply(0.0017)
print(count_threshold.getInfo())

occurrence_histogram = occurrence_water.reduceRegion(**{
    'reducer': ee.Reducer.autoHistogram(100,1),
    'geometry': image9.clip(roi).geometry(),
    'scale': 60,
})
def lookFrequency(array):
    counts = ee.Number(ee.List(array).get(1))
    a = ee.Algorithms.If(counts.gte(ee.Number(count_threshold)),ee.List(array).get(0))
    return a
occurrence_list = ee.List(occurrence_histogram.get('occurrence').getInfo())
occurrence_threshold = occurrence_list.map(lookFrequency).removeAll([None]).get(0)
print(occurrence_threshold.getInfo())

In [ ]:
Map.addLayer(water,{'palette':['blue']}, 'water')

occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
Map.addLayer(occurrence_corrected_water,{'palette': ['cyan']}, 'corrected waterbody')

enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
# Map.addLayer(enhanced_water,{'palette': ['blue']}, 'enhanced_water')

Map.addLayer(enhanced_water,{'palette':['D4D4D4','blue'],'min':0,'max':1}, 'enhanced_water')

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.812623, 34.507942, 91.5449, 35.6369]
raw_image = maskL8sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20180627').clip(roi))
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(landsat_images.mosaic().clip(raw_image.geometry()), region=region, vis_params=visParams)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'reference image', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.812623, 34.507942, 91.5449, 35.6369]
vis = {'palette':['D4D4D4','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(enhanced_water, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'classified image(AutomaticCorrection)', fontsize=20)

# 使用JRC数据集做交叉验证

## JRC

In [ ]:
# 使用JRC做验证
JRC_WaterClass = JRC_waterclass.rename('jrcwaterclass')
JRC_samplepoints = JRC_WaterClass.sample(**{
    'region': roi ,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('JRC total number of sample point:{}'.format(JRC_samplepoints.size().getInfo()))

waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',2))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

In [ ]:
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
permanent_water = waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
seasonal_water = ee.ImageCollection([waterfrequency.gte(0.25),waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
water_waterclass = ee.ImageCollection([permanent_water,seasonal_water,basemap]).sum()
# Map.addLayer(water_waterclass,{'palette':['white','red','blue'],'min':0,'max':2},"water waterclass")


# Overlay the points on the imagery to get training.
JRC_sample_point = water_waterclass.sampleRegions(**{
    'collection': JRC_samplepoints,
    'properties': ['jrcwaterclass'],
    'scale': 30
})
print(JRC_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,0,1]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum().rename('jrcwaterclass')
permanentwater = waterfrequency.gte(0.75).select('frequency').rename('waterclass')
JRC_samplepoints = JRC_waterclass.sample(**{
    'region': roi ,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('JRC total number of sample point:{}'.format(JRC_samplepoints.size().getInfo()))
waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))


JRC_sample_point = permanentwater.sampleRegions(**{
    'collection': JRC_samplepoints,
    'properties': ['jrcwaterclass'],
    'scale': 30
})
print(JRC_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

## Maryland

In [ ]:
# 使用Maryland做验证
Maryland_WaterClass = Maryland_waterclass.rename('Maryland_waterclass')
Maryland_samplepoints = Maryland_WaterClass.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('Maryland total number of sample point:{}'.format(Maryland_samplepoints.size().getInfo()))

waterclass = Maryland_samplepoints.filter(ee.Filter.eq('Maryland_waterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

waterclass = Maryland_samplepoints.filter(ee.Filter.eq('Maryland_waterclass',2))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

In [ ]:
# Overlay the points on the imagery to get training.
Maryland_sample_point = water_waterclass.sampleRegions(**{
    'collection': Maryland_samplepoints,
    'properties': ['Maryland_waterclass'],
    'scale': 30
})
print(Maryland_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = Maryland_sample_point.errorMatrix('Maryland_waterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
Maryland_WaterClass = Maryland_waterclass.remap([0,1,2],[0,0,1]).rename('Maryland_waterclass')
permanentwater = waterfrequency.gte(0.75).select('frequency').rename('waterclass')
Maryland_samplepoints = Maryland_WaterClass.sample(**{
    'region': roi ,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('Maryland total number of sample point:{}'.format(Maryland_samplepoints.size().getInfo()))
waterclass = Maryland_samplepoints.filter(ee.Filter.eq('Maryland_waterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))


Maryland_sample_point = permanentwater.sampleRegions(**{
    'collection': Maryland_samplepoints,
    'properties': ['Maryland_waterclass'],
    'scale': 30
})
print(Maryland_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = Maryland_sample_point.errorMatrix('Maryland_waterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

# 附录

In [ ]:
# Map.addLayer(sample_image.first().clip(roi),visParams,'image first')
permanent_image = sample_image.first().clip(roi).updateMask(permanentWaterExtent)
Map.addLayer(permanent_image,visParams,'permanent image')
permanent = permanentWaterExtent.updateMask(permanent_image.select('B2').gt(0))
Map.addLayer(permanent,{'palette':['white','blue'],'min':0,'max':1},'permanent')


season_image = sample_image.first().clip(roi).updateMask(seasonWaterExtent)
seasonal = seasonWaterExtent.updateMask(season_image.select('B2').gt(0))
Map.addLayer(seasonal,{'palette':['white','red'],'min':0,'max':1},'seasonal')

land_image = sample_image.first().clip(roi).updateMask(landExtent)
land = landExtent.updateMask(land_image.select('B2').gt(0)).remap([1],[0]).rename('waterclass')
Map.addLayer(land,{'palette':['white','cyan'],'min':0,'max':1},'land')

points_permanentwater = permanent.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 200,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print("permanent : {}".format(points_permanentwater.size().getInfo()))
points_seasonalwater = seasonal.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 200,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('seasonal : {}'.format(points_seasonalwater.size().getInfo()))

points_land = land.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 200,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('land : {}'.format(points_land.size().getInfo()))

In [ ]:
all_points = points_permanentwater.merge(points_seasonalwater).merge(points_land)
print(all_points.size().getInfo())
print(all_points.filter(ee.Filter.eq('waterclass',1)).size().getInfo())

In [ ]:
per_wuhan = landsat_images.mosaic().updateMask(permanentWaterExtent)
Map.addLayer(per_wuhan,visParams,"per wuhan")

In [ ]:
dataset = ee.Image('JRC/GSW1_2/YearlyHistory/2018')

visualization = {
  'bands': ['waterClass'],
  'min': 0.0,
  'max': 3.0,
  'palette': ['cccccc', 'ffffff', '99d9ea', '0000ff']
}
Map.addLayer(dataset.clip(roi), visualization, 'Water Class')

In [ ]:
# Maryland_waterclass显示
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4] 
vis = {'palette':['cccccc', '99d9ea', '0000ff'],'min':0,'max':2}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(Maryland_waterclass, region=region, vis_params=vis)
# add a colorbar to the map using the visualization params we passed to the map
# cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# ticks=range(0,50,10)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency(Maryland) in 2018', fontsize=20)

# JRC_waterclass
from matplotlib import pyplot as plt
from geemap import cartoee
import numpy as np
import matplotlib
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['cccccc', '99d9ea', '0000ff'],'min':0,'max':2}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(JRC_waterclass, region=region, vis_params=vis)
# cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency(JRC) in 2018', fontsize=20)

# index_waterclass
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['cccccc', '99d9ea', '0000ff'],'min':0,'max':2}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(index_waterclass, region=region, vis_params=vis)

# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency(INDEX) in 2018', fontsize=20)

In [ ]:
# seasonWaterExtent
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['cccccc', '99d9ea'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(seasonWaterExtent, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'seasonal waterbody extent in 2018', fontsize=20)

# permanentWaterExtent
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['cccccc', '0000ff'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentWaterExtent, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'permanent waterbody extent in 2018', fontsize=20)

# landExtent
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['white','cccccc'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(landExtent, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'land extent in 2018', fontsize=20)

In [ ]:
421.6652605777773
permanent = permanentWaterExtent.clip(image.geometry()).remap([0,1],[0,2]).rename('waterclass')
land = landExtent.clip(image.geometry())
season_image = image.clip(roi).updateMask(seasonWaterExtent)
awei = AWEIsh(season_image).select('AWEI')
intervalMean = awei.reduceRegion(**{
    'reducer': ee.Reducer.intervalMean(50,100), 
    'geometry': roi, 
    'scale': 30, 
    'bestEffort': True
})
season_class = awei.gt(ee.Image.constant(intervalMean.get('AWEI'))).rename('waterclass')
seasonalwater_Extent = ee.ImageCollection([season_class,basemap]).sum().remap([0,1],[0,2]).rename('waterclass').clip(image.geometry()) 
percentile_waterclass = ee.ImageCollection([permanent,seasonalwater_Extent,land]).sum().remap([0,1,2],[0,0,1]).rename('waterclass')